# Progetto NoSQL - Spotify
## Balducci Diego, Chiappella Alessandro, Panni Matteo, Scianca Davide

## Librerie

In [1]:
import requests
import base64

import json
from bson import json_util, ObjectId

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import pprint as pp

import pandas as pd
import numpy as np

import pymongo
import dns
from pymongo import MongoClient 

from neo4j import GraphDatabase

import os

In [2]:
import warnings
warnings.filterwarnings('ignore')

## Autorizzazioni per funzioni

In [3]:
# Authorization 
authUrl = "https://accounts.spotify.com/api/token"
authHeaders = {}
authData = {}

In [1]:
clientID = "clientID"
clientSecret = "clientSecret"

## Token

`getAccessTokne` è una funzione che utilizziamo per aver il permesso di scaricare i dati attarverso un `token`

In [5]:
def getAccessToken(clientID, clientSecret):
    message = f"{clientID}:{clientSecret}"
    messageBytes = message.encode('ascii')
    base64Bytes = base64.b64encode(messageBytes)
    base64Message = base64Bytes.decode('ascii')

    authHeaders['Authorization'] = f"Basic {base64Message}"
    authData['grant_type'] = "client_credentials"
    
    res = requests.post(authUrl, headers=authHeaders, data=authData)

    responseObject = res.json()
    
    accessToken = responseObject['access_token']
    
    print(accessToken)
    
    return accessToken

In [6]:
token = getAccessToken(clientID, clientSecret)

BQD5fyhX9Rr9_F_UPsPauiDDh3bzR7NS08UKKEkTzbr6lEdJkBFcnWKehaCS_a-_C16jfnYey89MC8fHnzIn-1QXRoUsImtq16HwJGVgj4bn_ugPl0w


## Autenticazione credenziali Spotipy

Con il `clientID` e il `clientSecret` eseguiamo l'autenticazione alla libreria di `spotipy`

In [7]:
client_credentials_manager = SpotifyClientCredentials(client_id=clientID, 
                                                      client_secret=clientSecret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

## Link

`userLink` è una lista in cui inseriamo i link del profilo degli utenti

In [8]:
userLink = [
    "https://open.spotify.com/user/gig1g5ugxxaprg5037v9s4e85?si=e05abb520be34654", # dave
    "https://open.spotify.com/user/s0llvygqb9553uvo8rh958ws8?si=a5dd3f37785a4b20", # diego
    "https://open.spotify.com/user/11126184093?si=01977197ccbf4e99", # teo
    "https://open.spotify.com/user/11147069621?si=EyE5ewVORWK6eNgXv6CxHw&utm", # ale
    "https://open.spotify.com/user/bea.passarino?si=RH1LQEW2QjSXp7fvcL6-rQ", # pandarino
    "https://open.spotify.com/user/no967zpi2pfq2zcwo3veh6y74?si=p0TaW7gQRn6_BBOCu4Hqkg", # chiara
    "https://open.spotify.com/user/31dafcka6zlofk7ltol5vr64nsgq?si=nziWAsN9QAGF6hyYbo5TUQ", # martol
    "https://open.spotify.com/user/21gljyydsnva2lf5m7zb4dyzi?si=73HfwnrVRI2mACWG9uD3Dw", # svngvlli
    "https://open.spotify.com/user/d8zrizl2vgb7bqwdhfgj2e9oh?si=6gBjjmsXQsGEkJg2O8ZTlA", # ousssssama
    "https://open.spotify.com/user/gamev3awajc0w7zm3un6oi7im?si=85a64cfebab44568", # Mehjibi
    "https://open.spotify.com/user/8an8d54s7xaucp6rhyvdm9odt?si=6f482c023c3b4e53", # pietro
    "https://open.spotify.com/user/ztvdly5rcltcv43e6gysceb2i?si=bycNzs0zSPWxy3VQ3q0oLg", # mattia
    "https://open.spotify.com/user/1186033663?si=e4a60ac0a27e4471", # Riccardo
    "https://open.spotify.com/user/nicholas.apuzzo1998?si=3943dd11fb4542e7", # Nicho
    "https://open.spotify.com/user/40typl19xf8hu1qq5nm7eiq6o?si=pF_5JuiUTE2wbG5M5WyHNw", # Ale Galli
    "https://open.spotify.com/user/l6r6ydnvf8hl1djc2aczqzu8f?si=P7oo7l9ETnCuK8arweFbDw", # Manuello
    "https://open.spotify.com/user/camicere?si=3IsttLanTHy-NhB2jDgwNg", # Cam
    "https://open.spotify.com/user/fossabmx?si=TFaQJjkSRN-UmosTQJI6GQ", # Fossa
    "https://open.spotify.com/user/davofuga?si=PTYVyRAlQ7GjDvlVtTiSQw", # Fughi
    "https://open.spotify.com/user/qhokpua1c15zyulkxf5yul0xa?si=PDkmJyyiTI2izEw0UpL8OA", # Anto
    "https://open.spotify.com/user/iw05lofaizn72mk59xuiteuae?si=1b1327486a694801", # Christian
    "https://open.spotify.com/user/elvica2000?si=xC09wL98TreKSITUJw8dTQ", # Thomas
    "https://open.spotify.com/user/maudevaglio?si=SQ9SxaekQkqO3tQbmtRPmA", # maude
    "https://open.spotify.com/user/matipelle?si=D3Q5p4XfQ-GDshX8KxUFZwSi",  #matipelle
    "https://open.spotify.com/user/31udo6usr5a2v3pfyql5d6phwygi?si=zoKwKC1-STiKcwn_thZkFg", # Naginni97
]

`playlistLink` è una lista in cui inseriamo i link delle playlist dei Wrapped degli utenti

In [9]:
playlistLink = [
    "https://open.spotify.com/playlist/5nfEnGujoYbIQwr3obusal?si=d91fbfbd170c40a2", # dave
    "https://open.spotify.com/playlist/1tBCOopkPhMy2qB4SPwsI2?si=6946af44fe05434b", # diego
    "https://open.spotify.com/playlist/3iHcsXyCmeaacp7nZo97kF?si=b02db0aa9f7c4599", # teo
    "https://open.spotify.com/playlist/1nkYSv9Crr6dNcktRGIyFn?si=ouROozJsSu6BjAS4DOH-KQ", # ale
    "https://open.spotify.com/playlist/6oqk5EJIxVO6Tj0dk9zOUx?si=-YF4tMb7QsiqAGoIK8WuJQ", # pandarino
    "https://open.spotify.com/playlist/1pvRY47Ho3XoiyFrAdutKn?si=-2sly5xLQgOwArwRssTPrw", # chiara
    "https://open.spotify.com/playlist/5VULiACGVkFQn0IOCyI0K1?si=SNrrCEwOSD6O59aws2bfEA", # martol
    "https://open.spotify.com/playlist/1hg8YLaTgyMdrR6yl8Xuq3?si=_kf-1mGOQs6T69EDLmqAew", # svngvlli
    "https://open.spotify.com/playlist/3SKtG7VKCLEb28Yt6hNIBk?si=1d893fb02ea84ce7", # ousssssama
    "https://open.spotify.com/playlist/132y10d7xgZr4AbY4zFRI5?si=20e6472102b14c45", # Mehjibi
    "https://open.spotify.com/playlist/442Ck0nGv4m0mgzisBVJqu?si=fc0d3b2358404ba5", # pietro
    "https://open.spotify.com/playlist/41tnvEo72msXMBsv8rFwQD?si=715ec792fb8d40f9", # mattia
    "https://open.spotify.com/playlist/6eoPrc7e9SCHwAELvmmm4j?si=OQtQbdqzQiWBACcp9_-MGA", # Riccardo
    "https://open.spotify.com/playlist/1MimFBHM1L2Zheybj8Cm89?si=f4c5182a84104c55", # Nicho
    "https://open.spotify.com/playlist/0JHkXdU9gAzwiNUbCZpEnM?si=ANWuKwJKQm-fpiyKYMBgRw", # Ale Galli
    "https://open.spotify.com/playlist/5o0mcvJ4MtfHgvFBIDNnhd?si=uvyD4HoNRkOZFwxisYfJmQ", # Manuello
    "https://open.spotify.com/playlist/6U2D5AMNCcAuvc0pe4YDTG?si=cx4Bu7vcSzWuJZxK1g36Eg", # Cam
    "https://open.spotify.com/playlist/5zF9oZWNUkp5zUwgOtRwIG?si=CCwnDhjgQLajChl-aLXR3g", # Fossa
    "https://open.spotify.com/playlist/1wFmsfFk0OVsf6GccKxtye?si=CBQselzWSlulEcrAZlM7qw", # Fughi
    "https://open.spotify.com/playlist/6ypFw761SWZoe2AtDIE7El?si=b8WX0ARDTBiayURiUFk-8Q", # Anto
    "https://open.spotify.com/playlist/09oiswzRqcof7Ht8uqTMDj?si=d8610fe402b9410b", # Christian
    "https://open.spotify.com/playlist/13GuODVRIIRKrDYPumF5CF?si=GldmZWM7QYKLcLb8kOcdgQ", # Thomas
    "https://open.spotify.com/playlist/1uUb14vhSuHrK1yEjEVKs3?si=c4bY7MygTO6_OOWL80Bugw", # maude
    "https://open.spotify.com/playlist/2AXwY01SI0AEFI7tsObe7F?si=Z7XXEzV3SrCfegY8oT-D7Q&nd=1", # matipelle
    "https://open.spotify.com/playlist/0tzApf6Md0ir30EKtBifN5?si=2CDBq0uFSf-7w6RlRwQpHw", # Naginni97
]

Verifica dell'omogenità dei delle lunghezze delle due liste

In [10]:
print(f'''userLink: {len(userLink)}
playlistLink: {len(playlistLink)}''')

userLink: 25
playlistLink: 25


## Extract_multiple_ids

La funzione `extract_multiple_ids` permette di estrarre:
1. playlistID
2. playlistURI
3. userID

attraverso `split` all-interno di un ciclo `for` che legge le liste `userLink` e `playlistLink` e vengono inseriti nella lista `extracted_ids`

In [11]:
def extract_multiple_ids(playlist_links, user_links):
    extracted_ids = []
    
    for playlist_link, user_link in zip(playlist_links, user_links):
        playlistID = playlist_link.split("/")[-1]
        playlistURI = playlist_link.split("/")[-1].split("?")[0]
        userID = user_link.split("/")[-1].split("?")[-1].split("=")[-1]
        extracted_ids.append((userID, playlistID, playlistURI))
        
    return extracted_ids

In [12]:
extracted_ids = extract_multiple_ids(playlistLink, userLink)

In [13]:
len(extracted_ids)

25

## Get_playlist_tracks

La funzione `get_playlist_tracks` estrae attraverso `sp.user_playlist_tracks()` le tracce all'interno delle playlist passate.

La funzione ritorna `tracks`, una lista che contiene tutte le tracce.

In [14]:
def get_playlist_tracks(username,playlist_id):
    # Utilizza l'API Web Spotify per recuperare il primo set di tracce dalla playlist specificata
    results = sp.user_playlist_tracks(username,playlist_id)
    # Immagazzina le tracce in una lista
    tracks = results['items']
    
    # Controlla se c'è un'altra pagina di tracce
    while results['next']:
        # Recupera la pagina successiva di tracce
        results = sp.next(results)
        # Aggiungi alla la lista 'tracks' le tracce
        tracks.extend(results['items'])
    
    return tracks

## Get_track_features

La funzione `get_track_features` estrae attraverso `sp.audio_features(batch)` le features (le features sono tutte le caratteristiche delle canzoni) all'interno degli id delle tracce passati.

La funzione ritorna `features`, una lista che contiene tutte le features delle tracce.

>La differenza tra `append` e `extend` è che:
 - "append" aggiunge un singolo elemento alla fine di una lista, 
 - mentre "extend" aggiunge gli elementi di un'altra lista alla fine di una lista esistente.

In [15]:
def get_track_features(track_ids):
    features = []
    
    for i in range(0, len(track_ids), 50):
        # Ottieni un gruppo di ID di tracce dall'iterazione corrente del ciclo
        batch = track_ids[i:i+50]
        # Utilizzare l'API di Spotify per recuperare le caratteristiche audio per il gruppo corrente di ID di tracce
        results = sp.audio_features(batch)
        # Aggiungi alla la lista 'features' le caratteristiche audio dal gruppo corrente
        features.extend(results)     
        
    return features

Inserimento delle tracce all'interno della lista `tracks_list` richiamando la funzione `get_playlist_tracks`.

`tracks_list` è una lista che contiene liste di tutti gli utenti che a loro volta sono liste di dizionari (ogni dizionario corrisponde a una canzone con le sue generalità)

In [16]:
tracks_list = []
for userID, playlistID, playlistURI in extracted_ids:
    tracks = get_playlist_tracks(userID, playlistURI)
    tracks_list.append(tracks)

KeyboardInterrupt: 

In [ ]:
lista = np.arange(len(tracks_list))
for i in lista:
    print(f"utente {i+1}: {len(tracks_list[i])} tracce")

Inserimento delle features delle tracce all'interno della lista `audio_features_list` richiamando la funzione `get_track_features`.

`audio_features_list` è una lista che contiene liste di tutti gli utenti che a loro volta sono liste di dizionari (ogni dizionario corrisponde le features di una canzone)

In [ ]:
audio_features_list = []

# doppio ciclo for per accedere al valore del field ['track']['uri'] -> l'uri è un identificativo della traccia. es: 'spotify:track:7iLuBTHJSXM2HalKHFqEEy'
track_ids = [d['track']['uri'] for sublist in tracks_list for d in sublist]
audio_features = get_track_features(track_ids)
audio_features_list.append(audio_features)

## Trasformazioni dati in JSON

La funzione `save_to_json` effettua il savaltaggio di una lista in un file JSON

In [ ]:
def save_to_json(obj, filename):
    with open(filename, 'w') as f:
        json.dump(obj, f)

### totale_track_101 JSON

In [ ]:
track_list_json = save_to_json(tracks_list, "tracks_list.json")

### totale_audio_features JSON

In [ ]:
save_to_json(audio_features_list, "audio_features_list.json")

## MongoDB

Autenticazione al server di `MongoDB` attraverso le credenziali

In [ ]:
password = "s5ya6KD41Gs5EBks"
MONGODB_URL_STRING = "mongodb+srv://davide_scianca:" + password + "@cluster0.zaeh3wk.mongodb.net/?retryWrites=true&w=majority"

In [ ]:
client = MongoClient(MONGODB_URL_STRING)
print(client)

### Visualizzazione db

In [ ]:
def display_db(client):
    db = client.list_database_names()
    print(db)

In [ ]:
display_db(client)

### Creazione db

In [ ]:
db = client.lab_spoty
db

### Visualizzazione collection 
prima dell'inserimento

In [ ]:
def display_collections(db):
    collections = db.list_collection_names()
    print(collections)

In [ ]:
display_collections(db)

### Creazione collection

In [ ]:
tracks_collection = db.track_list
audio_features_collection = db.features_list

### Caricamento documenti su Mongo db

In [ ]:
def insert_data_to_mongodb(json_file, collection):
    """json_file: the name of the JSON file that you want to read data from.
    collection: the MongoDB collection that you want to insert the data into."""
    
    with open(json_file, "r") as data_file:
        data = json.load(data_file)
        
    client = MongoClient()
    db = client.mydb
    
    # doppio ciclo for per poter accedere ai dizionari, visto sono contenuti all'interno di due liste
    for outer_list in data:
    # Itera attraverso ogni lista interna
        for inner_list in outer_list:
        # Inserisci il dizionario come documento nella raccolta
            collection.insert_one(inner_list)
        
    print("Data inserted to MongoDB")

### Eliminazione documenti su Mongo db

La funzione `delete_all_data` è stata creata per velocizzare l'eliminazione dei dati e il succesivo ricaricamento con i dati aggiornati.

In [ ]:
def delete_all_data(collection):
    """collection: the MongoDB collection that you want to delete the data into."""
    
    collection.delete_many({})
    
    print("All data deleted from collection.")

#### tracks_collection

In [ ]:
# insert_data_to_mongodb("tracks_list.json", tracks_collection)

In [ ]:
# delete_all_data(tracks_collection)

#### totale_audio_features

In [ ]:
# insert_data_to_mongodb('audio_features_list.json', audio_features_collection)

In [ ]:
# delete_all_data(audio_features_collection)

### Visualizzazione collection
dopo l'inserimento

In [ ]:
display_collections(db)

### Conteggio documenti

In [ ]:
def count_documents_all_collections(db):
    collections = db.list_collection_names()
    for collection in collections:
        count = db[collection].count_documents({})
        print(f"Number of documents in {collection} collection: \n\t{count}\n")

In [ ]:
count_documents_all_collections(db)

### Lookup

La funzione `lookupCollection` permette di eseguire il merge attraverso `uri` tra la lista `tracks_list` e `playlistLink`

In [ ]:
def lookupCollection(db, main_collection, join_collection, main_field, join_field, as_name):
    """db: the MongoDB database that you want to perform the join on.
    main_collection: the main collection that you want to join with the join_collection.
    join_collection: the collection that you want to join to the main collection.
    main_field: the field from the main collection to be used for the join.
    join_field: the field from the join collection to be used for the join.
    as_name: the name of the new array field that contains the joined documents"""
    
    lookup_query = [{'$lookup' : 
                        {'from': join_collection,
                         'localField': main_field,
                         'foreignField': join_field,
                         'as': as_name }
                    }]
    
    joined_docs = db[main_collection].aggregate(lookup_query)
    
    return list(joined_docs)

#### Lookup tra Uri di tracks_collection e audio_features_collection

In [ ]:
tracks_features_merged = lookupCollection(db, 'track_list', 'features_list', 'track.uri', 'uri', 'audioFeatures')

La lista `listanumeri` serve in seguito per estrarre i dati dalla lista di dizionari

In [ ]:
listanumeri = np.arange(0, len(tracks_features_merged))

#### Cancellazione degli ObjectID e creazione di un file JSON
Serve fare questo passaggio perchè i file JSON non possono contenere gli `ObjectID` (ID univoco che mongo assegna per ogni document)

In [ ]:
for num in listanumeri:
    if 'audioFeatures' in tracks_features_merged[num]:
        for feature in tracks_features_merged[num]['audioFeatures']:
            if '_id' in feature:
                del feature['_id']
        if '_id' in tracks_features_merged[num]:
            del tracks_features_merged[num]['_id']

In [ ]:
def parse_json(data):
    return json.loads(json_util.dumps(data))

In [ ]:
tracks_features_merged = parse_json(tracks_features_merged)
len(tracks_features_merged)

Salvataggio in formato JSON del risultato del merge fatto su Mongo

In [ ]:
save_to_json(tracks_features_merged, "tracks_features_merged.json")

## Preparazione ai dati per Neo4j
Per ogni tipologia di nodi che si vuole caricare su Neo4j si crea un csv.

La funzione `df_to_csv` per prima cosa converte i `Dataframe` creati per il caricamento dei nodi su `Neo4j` in `csv`, e li carica in una cartella specifica dopo esser stata creata.

In [ ]:
def df_to_csv(df, name_csv):
    path = 'C:\\Users\\david\\OneDrive\\Documenti\\TTF\\ANNO 2022-2023\\Progetto Spoty NoSQL\\prova advance\\df_to_csv\\'
    
    if not os.path.exists(path):
        os.makedirs(path)
        
    file_path = os.path.join(path, name_csv)
    
    df.to_csv(file_path, index=False)

### Utenti

La lista `listaNumStep` serve per estrarre un singolo id per ogni utente

In [ ]:
listaNumStep = np.arange(0, len(tracks_features_merged), 101)

#### Creazione delle variabili idUser e song_id_groups

Estrazione in `idUser` degli id degi utenti

In [ ]:
idUser = [tracks_features_merged[num]['added_by']['id'] for num in listaNumStep]

Creazione di gruppi di 101 canzoni divise

In [ ]:
group_size = 101
song_ids = [x["track"]["id"] for x in tracks_features_merged]
song_id_groups = [song_ids[i:i+group_size] for i in range(0, len(song_ids), group_size)]

In [ ]:
utenti_dict = [{"name": "Dave", "id":"", "track":""},
               {"name": "Diego", "id":"", "track":""},
               {"name": "Teo", "id":"", "track":""},
               {"name": "Ale", "id":"", "track":""},
               {"name": "Pandarino", "id":"", "track":""},
               {"name": "Chiara", "id":"", "track":""},
               {"name": "Martol", "id":"", "track":""},
               {"name": "Svngvlli", "id":"", "track":""},
               {"name": "Ousssssama", "id":"", "track":""},
               {"name": "Mehjibi", "id":"", "track":""},
               {"name": "Pietro", "id":"", "track":""},
               {"name": "Mattia", "id":"", "track":""},
               {"name": "Riccardo", "id":"", "track":""},
               {"name": "Nicho", "id":"", "track":""},
               {"name": "Ale Galli", "id":"", "track":""},
               {"name": "Manuello", "id":"", "track":""},
               {"name": "Cam", "id":"", "track":""},
               {"name": "Fossa", "id":"", "track":""},
               {"name": "Fughi", "id":"", "track":""},
               {"name": "Anto", "id":"", "track":""},
               {"name": "Christian", "id":"", "track":""},
               {"name": "Thomas", "id":"", "track":""},
               {"name": "Maude", "id":"", "track":""},
               {"name": "matipelle", "id":"", "track":""},
               {"name": "Naginni97", "id":"", "track":""},
              ]

inserimento delle variabili all'interno del dizionario `utenti_dict`

In [ ]:
for i in range(len(idUser)):
    utenti_dict[i]['id'] = idUser[i]
    utenti_dict[i]['track'] = song_id_groups[i]

Conversione del dizionario `utenti_dict` in Dataframe

In [ ]:
df_users = pd.DataFrame(utenti_dict)
df_users

Conversione del Dataframe in csv

In [ ]:
df_to_csv(df_users, "df_users.csv")

### Traccia

Inserimento all'interno delle liste: 
- danceability
- energy
- key
- loudness
- mode
- speechiness
- acousticness
- instrumentalness
- liveness
- valence
- tempo
- duration_ms
- nameAlbumTr
- idAlbumTr

i valori attraverso un ciclo for che itera all'interno della lista di Dizionari

All'interno di `tracks_artists` vengono aggiunti:
- tracks
- idTracks
- artist
- idArtist

if e elif servono per gestire il caso in cui una traccia sia cantata da più artisti

In [ ]:
tracks_artists =[]
for num in listanumeri:
    if len(tracks_features_merged[num]['track']['artists']) == 1:
        tracks_artists.append({'tracks':tracks_features_merged[num]['track']['name'],
                               'idTracks':tracks_features_merged[num]['audioFeatures'][0]['id'],
                               'artist': tracks_features_merged[num]['track']['artists'][0]['name'],
                               'idArtist': tracks_features_merged[num]['track']['artists'][0]['id']})

    elif len(tracks_features_merged[num]['track']['artists']) > 1:
        tracks_artists.append({'tracks':tracks_features_merged[num]['track']['name'],
                               'idTracks':tracks_features_merged[num]['audioFeatures'][0]['id'],
                               'artist': [i['name'] for i in tracks_features_merged[num]['track']['artists']],
                               'idArtist': [i['id'] for i in tracks_features_merged[num]['track']['artists']]})

danceability = []; energy = []; key = []; loudness = [];
mode = []; speechiness = []; acousticness = []; instrumentalness = []; liveness = []
valence = []; tempo = []; duration_ms = []; nameAlbumTr = []; idAlbumTr = [];

for num in listanumeri:
    nameAlbumTr.append(tracks_features_merged[num]['track']['album']['name'])
    idAlbumTr.append(tracks_features_merged[num]['track']['album']['id'])
    danceability.append(tracks_features_merged[num]['audioFeatures'][0]['danceability'])
    energy.append(tracks_features_merged[num]['audioFeatures'][0]['energy'])
    key.append(tracks_features_merged[num]['audioFeatures'][0]['key'])
    loudness.append(tracks_features_merged[num]['audioFeatures'][0]['loudness'])
    mode.append(tracks_features_merged[num]['audioFeatures'][0]['mode'])
    speechiness.append(tracks_features_merged[num]['audioFeatures'][0]['speechiness'])
    acousticness.append(tracks_features_merged[num]['audioFeatures'][0]['acousticness'])
    instrumentalness.append(tracks_features_merged[num]['audioFeatures'][0]['instrumentalness'])
    liveness.append(tracks_features_merged[num]['audioFeatures'][0]['liveness'])
    valence.append(tracks_features_merged[num]['audioFeatures'][0]['valence'])
    tempo.append(tracks_features_merged[num]['audioFeatures'][0]['tempo'])
    duration_ms.append(tracks_features_merged[num]['audioFeatures'][0]['duration_ms'])

Creazione di due liste:
- `lista_chiavi_Tr` che contiene le chiavi di un dizionario che creiamo nella cella successiva
- `lista_valori_Tr` che contiene le variabili della liste create

In [ ]:
lista_chiavi_Tr = ['nameAlbumTr', 'idAlbumTr', 'danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo','duration_ms']
lista_valori_Tr = [nameAlbumTr, idAlbumTr, danceability, energy, key, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence, tempo, duration_ms]

Creazione del dizionario

In [ ]:
properties_tracks = {lista_chiavi_Tr[i]:lista_valori_Tr[i] for i in range(len(lista_chiavi_Tr))}

Conversione della lista `track_artists` e del dizionario `properties_tracks` in un Dataframe

In [ ]:
df_tracks = pd.concat([pd.DataFrame(tracks_artists), pd.DataFrame(properties_tracks)], axis=1)
df_tracks.head()

Conversione del Dataframe in csv

In [ ]:
df_to_csv(df_tracks, "df_tracks.csv")

#### lista utenti che hanno ascoltato la stessa canzone
questa è la soluzione nel Notebook di chi ha ascolato la stessa canzone. Non è inclusa all'interno del csv.

In [ ]:
track = []; idTrack = []; users = [];

for num in listanumeri:
    track.append(tracks_features_merged[num]['track']['name'])
    idTrack.append(tracks_features_merged[num]['track']['name'])
    users.append(tracks_features_merged[num]['added_by']['id'])

track_user = {'Tracks': track, 'idTracks': idTrack, 'Users': users}
df_users_tracks = pd.DataFrame(track_user)

In [ ]:
pd.options.display.max_rows = None

In [ ]:
# groupby per traccia creando una lista degli utenti che l'hanno ascoltata
df_users_tracks_agg = df_users_tracks.groupby('Tracks')['Users'].agg(list).reset_index()

# selezione delle canzoni che sono state ascoltate da più utenti
more_user = df_users_tracks_agg['Users'].apply(lambda x: len(x) > 1)

# creazione di una lista per contenere gli utenti che hanno ascolato la stessa canzone 
more_user_index = more_user[more_user == True].index.tolist()
df_users_tracks_agg = df_users_tracks_agg.iloc[more_user_index]

# conversione dell'id con il nome inserito all'interno del dizionario utenti_dict
df_users_tracks_agg['name'] = df_users_tracks_agg['Users'].apply(lambda x: [d['name'] for d in utenti_dict if d['id'] in x])

df_users_tracks_agg.head()

Conversione del Dataframe in csv

In [ ]:
df_to_csv(df_users_tracks_agg, "df_users_tracks_agg.csv")

### Artist

Essendo che la liibreria `spotipy` non fornisce il genere musicale degli artisti, la funzione `get_artist_data` fornisce dall'Endpoint dell'API un dizionario contenente il nome, l'id e i generi dell'artista.

In [ ]:
def get_artist_data(artist_ids, client_id, client_secret):
    # Get an access token
    response = requests.post("https://accounts.spotify.com/api/token", {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    })
    access_token = response.json()["access_token"]

    # Set the authorization header
    headers = {
        "Authorization": f"Bearer {access_token}"
    }

    artist_data = []
    for artist_id in artist_ids:
        try:
            # Make a request to the Spotify API to get an artist
            response = requests.get(f"https://api.spotify.com/v1/artists/{artist_id}", headers=headers)

            # Get the artist data
            artist = response.json()

            # Append the artist name, ID, and primary genre to the artist_data list
            artist_data.append({
                "name": artist["name"],
                "id": artist["id"],
                "genre": artist["genres"]
            })
        except:
            # Handle the error here, for example, by logging it
            print(f'Error occured while getting artist {artist_id}')
            continue
    return artist_data

In [ ]:
art = []
idArt = []
for num in listanumeri:
    for num1 in tracks_features_merged[num]['track']['artists']:
        art.append(num1['name'])
        idArt.append(num1['id'])

In [ ]:
art = [x for i, x in enumerate(art) if i == art.index(x)]
idArt = [x for i, x in enumerate(idArt) if i == idArt.index(x)]

len(art)

Conversione del dizionario `artist_data` in Dataframe

In [ ]:
artist_data = get_artist_data(idArt, clientID, clientSecret)

df_artist = pd.DataFrame(artist_data)
df_artist.head()

#### artisti senza genere
Visto che alcuni artisti non hanno un genere musicale definito la cella della colonna `genre` corrispondente è stata modificata inserendo la stringa `Without Genre`

In [ ]:
empty_lists = df_artist['genre'].apply(lambda x: len(x) == 0)

empty_lists_index = empty_lists[empty_lists == True].index.tolist()

df_artist['genre'] = df_artist['genre'].apply(lambda x: x if len(x) > 0 else ["Without Genre"])

Conversione del Dataframe in csv

In [ ]:
df_to_csv(df_artist, "df_artist.csv")

### Album
Inserimento all'interno delle liste: 
- nameAlbum
- idAlbum
- typeAlbum
- totalTracks
- releaseDate

i valori attraverso un ciclo for che itera all'interno della lista di Dizionari

All'interno di `album_artists` vengono aggiunti:
- artist
- idArtist


if e elif servono per gestire il caso in cui un album appartenga a più artisti

In [ ]:
nameAlbum = []; idAlbum = []; typeAlbum = [];
totalTracks = []; releaseDate = []; 

for num in listanumeri:
    nameAlbum.append(tracks_features_merged[num]['track']['album']['name'])
    idAlbum.append(tracks_features_merged[num]['track']['album']['id'])
    typeAlbum.append(tracks_features_merged[num]['track']['album']['album_type'])
    totalTracks.append(tracks_features_merged[num]['track']['album']['total_tracks'])
    releaseDate.append(tracks_features_merged[num]['track']['album']['release_date'])
    
album_artists = []
for num in listanumeri:
    if len(tracks_features_merged[num]['track']['album']['artists']) == 1:
        album_artists.append({'artist': tracks_features_merged[num]['track']['artists'][0]['name'],
                               'idArtist': tracks_features_merged[num]['track']['artists'][0]['id']})

    elif len(tracks_features_merged[num]['track']['album']['artists']) > 1:
        album_artists.append({'artist': [i['name'] for i in tracks_features_merged[num]['track']['artists']],
                               'idArtist': [i['id'] for i in tracks_features_merged[num]['track']['artists']]})

Creazione di due liste:
- `lista_chiavi_Tr` che contiene le chiavi di un dizionario che creiamo nella cella successiva
- `lista_valori_Tr` che contiene le variabili della liste create

In [ ]:
lista_chiavi_Al = ['nameAlbum','idAlbum','typeAlbum','totalTracks','releaseDate']
lista_valori_Al = [nameAlbum, idAlbum, typeAlbum, totalTracks, releaseDate]

album_data = {lista_chiavi_Al[i]:lista_valori_Al[i] for i in range(len(lista_chiavi_Al))}

Conversione della lista `album_artists` e del dizionario `album_data` in un Dataframe

In [ ]:
df_album = pd.concat([pd.DataFrame(album_data), pd.DataFrame(album_artists)], axis=1)
df_album.head()

Conversione del Dataframe in csv

In [ ]:
df_to_csv(df_album, "df_album.csv")

### Genre
Vengono presi tutti i generi di tutte liste all'interno della colonna `genre` del df `df_artist` e vengono rimossi i duplicati

In [ ]:
generi = df_artist['genre'].tolist()

# appiattimento lista generi
generi = [el for sub in generi for el in sub]

# rimozione elementi duplicati
generi = [x for i, x in enumerate(generi) if i == generi.index(x)]

In [ ]:
series_generi = pd.Series(generi, name="genres")
series_generi.head()

In [ ]:
series_generi.shape

Conversione della Series in csv

In [ ]:
df_to_csv(series_generi, "series_generi.csv")

## Neo4j

### Autenticazione
Autenticazione a Neo4j con le credenziali di neo4j

In [ ]:
uri = "neo4j+s://e1f35a86.databases.neo4j.io"
user = "neo4j"
password = "B86HHTw6sSK_OVsCf0Aupckr5B4KRBMT_qqs00LErFk"

In [ ]:
neo4jInstance = GraphDatabase.driver(uri, auth=(user, password))

#### run Query

La funzione `runNeo4jQuery` esegue la query inserita in formato stringa all'interno di una variabile

In [ ]:
def runNeo4jQuery(query):
    with neo4jInstance.session(database="neo4j") as session:
        result = session.run(query)
        #pp.pprint(result.__dict__)
        for row in result:
            print(str(row))
        session.close()

#### Pulizia dati
La funzione `clean_column` grazie ad una regex elimina i caratteri che non sono ammessi su neo4j visto che sono all'interno delle colonne `track`, `artist` e `album`

In [ ]:
def clean_column(df, column):
    import string
    df[column] = df[column].apply(lambda x: x.translate(str.maketrans("", "", ":.[](){}'\"*?/\\")))
    return df

#### eliminazione di tutti i nodi e di tutte le relazioni

La seguente query è stata creata per velocizzare l'eliminazione dei dati e il succesivo ricaricamento con i dati aggiornati.

In [ ]:
query_delete = "MATCH (n) DETACH DELETE n"
# runNeo4jQuery(query_delete)

## Caricamento Nodi

In [ ]:
def import_csv(file_path):
    data = pd.read_csv(file_path)
    return data

### Utenti

In [ ]:
users_path = "C:\\Users\\david\\OneDrive\\Documenti\\TTF\\ANNO 2022-2023\\Progetto Spoty NoSQL\\prova advance\\df_to_csv\\df_users.csv"
users_csv = import_csv(users_path)

In [ ]:
for index, row in users_csv.iterrows():
    name = row["name"]
    id = row["id"]
    track = row["track"]
    query_node_users = f'CREATE (u:User {{ nameUser: "{name}", id: "{id}", track: "{track}"}})'
    
    # runNeo4jQuery(query_node_users)

### Traccia

In [ ]:
tracks_path = "C:\\Users\\david\\OneDrive\\Documenti\\TTF\\ANNO 2022-2023\\Progetto Spoty NoSQL\\prova advance\\df_to_csv\\df_tracks.csv"
tracks_csv = import_csv(tracks_path)

In [ ]:
tracks_csv = clean_column(tracks_csv, "tracks")
tracks_csv = clean_column(tracks_csv, "artist")
tracks_csv = clean_column(tracks_csv, "nameAlbumTr")

In [ ]:
for index, row in tracks_csv.iterrows():
    tracks = row["tracks"]
    idTracks = row["idTracks"]
    artist = row["artist"]
    idArtist = row["idArtist"]
    nameAlbum = row["nameAlbumTr"]
    idAlbum = row["idAlbumTr"]
    danceability = row["danceability"]
    key = row["key"]
    loudness = row["loudness"]
    mode = row["mode"]
    speechiness = row["speechiness"]
    acousticness = row["acousticness"]
    instrumentalness = row["instrumentalness"]
    liveness = row["liveness"]
    valence = row["valence"]
    tempo = row["tempo"]
    duration_ms = row["duration_ms"]    
    
    query_node_tracks = f'CREATE (t:Track {{nameTrack: "{tracks}", idTracks: "{idTracks}", artist: "{artist}", \
    idArtist: "{idArtist}", nameAlbum: "{nameAlbum}", idAlbum: "{idAlbum}", danceability: {float(danceability)}, \
    key: {int(key)}, loudness: {float(loudness)}, mode: {int(mode)}, speechiness: {float(speechiness)}, acousticness: {float(acousticness)}, \
    instrumentalness: {float(instrumentalness)}, liveness: {float(liveness)}, valence: {float(valence)}, tempo: {float(tempo)}, \
    duration_ms: {int(duration_ms)} }} )'

    # runNeo4jQuery(query_node_tracks)

### Artist

In [ ]:
artist_path = "C:\\Users\\david\\OneDrive\\Documenti\\TTF\\ANNO 2022-2023\\Progetto Spoty NoSQL\\prova advance\\df_to_csv\\df_artist.csv"
artist_csv = import_csv(artist_path)

In [ ]:
artist_csv = clean_column(artist_csv, "name")

In [ ]:
for index, row in artist_csv.iterrows():
    name = row["name"]
    id = row["id"]
    genre = row["genre"]
    
    query_node_artists = f'CREATE (a:Artist {{nameArtist: "{name}", id: "{id}", genre: "{genre}"}})'
    
    # runNeo4jQuery(query_node_artists)

### Album

In [ ]:
album_path = "C:\\Users\\david\\OneDrive\\Documenti\\TTF\\ANNO 2022-2023\\Progetto Spoty NoSQL\\prova advance\\df_to_csv\\df_album.csv"
album_csv = import_csv(album_path)

In [ ]:
album_csv = clean_column(album_csv, "nameAlbum")
album_csv = clean_column(album_csv, "artist")

In [ ]:
album_csv['releaseDate'] = pd.to_datetime(album_csv['releaseDate']).dt.date

In [ ]:
for index, row in album_csv.iterrows():
    nameAlbum = row["nameAlbum"]
    idAlbum = row["idAlbum"]
    typeAlbum = row["typeAlbum"]
    totalTracks = row["totalTracks"]
    releaseDate = row["releaseDate"]
    artist = row["artist"]
    idArtist = row["idArtist"]

    query_node_albums = f'CREATE (al:Album {{nameAlbum: "{nameAlbum}", idAlbum: "{idAlbum}", typeAlbum: "{typeAlbum}",\
    totalTracks: "{int(totalTracks)}", releaseDate: "{(releaseDate)}", artist: "{artist}", idArtist: "{idArtist}"}})'

    # runNeo4jQuery(query_node_albums)

### Genre

In [ ]:
genre_path = "C:\\Users\\david\\OneDrive\\Documenti\\TTF\\ANNO 2022-2023\\Progetto Spoty NoSQL\\prova advance\\df_to_csv\\series_generi.csv"
genre_csv = import_csv(genre_path)

In [ ]:
for index, row in genre_csv.iterrows():
    genres = row["genres"]
    
    query_node_genres = f'CREATE (g:Genre {{genres: "{genres}"}})'
    
    # runNeo4jQuery(query_node_genres)

## Creazione Relazioni tra i nodi

### Users -> Tracks

In [ ]:
users_tracks = 'MATCH (u:User), (t:Track) \
WHERE u.track CONTAINS t.idTracks \
CREATE (u)-[r:LISTENED_TO]->(t)'

# runNeo4jQuery(users_tracks)

### Tra Tracks -> Album

In [ ]:
tracks_album = 'MATCH (t:Track), (al:Album) \
WHERE t.idAlbum = al.idAlbum \
MERGE (t)-[r:PART_OF]->(al)'

# runNeo4jQuery(tracks_album)

### Tracks -> Artists

In [ ]:
tracks_artists = 'MATCH (t:Track), (a:Artist) \
WHERE t.idArtist = a.id \
CREATE (t)-[r:PERFORMED_BY]->(a)'

# runNeo4jQuery(tracks_artists)

### Artist -> Genres

In [ ]:
artists_genres = 'MATCH (a:Artist), (g:Genre) \
WHERE a.genre CONTAINS g.genres \
CREATE (a)-[r:BELONGS_TO]->(g)'

# runNeo4jQuery(artists_genres)

### Artist -> Artist

In [ ]:
artists_artist = '''MATCH (a1:Artist),(a2:Artist)
WHERE a1 <> a2
WITH a1, a2
MATCH (t:Track)
WHERE t.artist CONTAINS a1.nameArtist AND t.artist CONTAINS a2.nameArtist
CREATE (a1)-[r:COLLABORATED_WITH]->(a2)'''

# runNeo4jQuery(artists_artist)

### Album -> Artist

In [ ]:
album_artist = 'MATCH (al:Album), (a:Artist) \
WHERE al.idArtist = a.id \
CREATE (al)-[r:BY_ARTIST]->(a)'

# runNeo4jQuery(album_artist)

## Query

### 1) Stabilire quale sia la canzone più presente nei Wrapped 2022: 

In [ ]:
def most_listened_song(tx):
    """funzione per trovare la canzone più ascoltata"""
    
    result = tx.run('''MATCH (t:Track)<-[:LISTENED_TO]-(u:User) 
RETURN t.nameTrack
LIMIT 1''')
    return result.single()

In [ ]:
with neo4jInstance.session() as session:
    result = session.read_transaction(most_listened_song)
    print(f"Canzone più ascoltata: {result['t.nameTrack']}")

### 2) Stabilire quali siano le canzoni in comune tra gli utenti, quanti ascoltatori hanno avuto e quali sono:

In [ ]:
with neo4jInstance.session() as session:
    result = session.run("MATCH (u1:User)-[:LISTENED_TO]->(t:Track)<-[:LISTENED_TO]-(u2:User) \
                    WHERE u1 <> u2 \
                    RETURN t.nameTrack, COUNT(DISTINCT u1) as numUsers, COLLECT(DISTINCT u1.nameUser) as commonUsers \
                    ORDER BY numUsers DESC, SIZE(commonUsers) DESC")
    
    for record in result:
        print(f'''Track: {record["t.nameTrack"]} 
            num: {record["numUsers"]} 
            User: {record["commonUsers"]}\n''')

### 3) Stabilire quale sia l'artista più presente nei Wrapped 2022:

In [ ]:
def most_listened_artist(tx):
    """funzione per trovare l'artista più ascoltato"""
    
    result = tx.run('''MATCH (u:User)-[:LISTENED_TO]->(t:Track)-[:PERFORMED_BY]->(a:Artist)
WITH a.nameArtist as artist, count(a) as count 
ORDER BY count DESC 
LIMIT 1
RETURN artist''')
    return result.single()

In [ ]:
with neo4jInstance.session() as session:
    result = session.read_transaction(most_listened_artist)
    print("Artista più ascoltato", result["artist"])

### 4) Per ogni utente, stabilire quale sia l'artista più presente nel proprio Wrapped:

In [ ]:
def get_all_users(tx):
    """funzione per ottenere tutti gli utenti"""
    
    result = tx.run("MATCH (u:User) RETURN u.nameUser as nameUser")
    return [record["nameUser"] for record in result]

In [ ]:
def most_listened_artist(tx, nameUser):
    """funzione per trovare il genere più ascoltato da un utente"""
    
    result = tx.run("MATCH (u:User)-[:LISTENED_TO]->(t:Track)-[:PERFORMED_BY]->(a:Artist)-[:BELONGS_TO]->(g:Genre) \
                    WHERE u.nameUser = $nameUser \
                    RETURN a.nameArtist as artist, count(a) as count \
                    ORDER BY count DESC \
                    LIMIT 1", nameUser=nameUser)
    return result.single()

In [ ]:
with neo4jInstance.session() as session:
    # ottieni tutti gli utenti
    users = session.read_transaction(get_all_users)
    
    # per ogni utente, trova l'artista più ascoltato
    for user in users:
        result = session.read_transaction(most_listened_artist, user)
        print(f'''Utente: {user} 
        Artista più ascoltato: {result["artist"]}\n''')

### 5) Stabilire per ogni artista quante collaborazioni ha avuto e con quali artisti all’interno delle tracce del Wrapped:

In [ ]:
with neo4jInstance.session() as session:
    result = session.run('''MATCH (a1:Artist)-[r:COLLABORATED_WITH]->(a2:Artist)
WITH a1.nameArtist as name, 
COLLECT(DISTINCT a2.nameArtist) as collaborators, 
SIZE(COLLECT(DISTINCT a2.nameArtist)) as collaborations
RETURN name, collaborations, collaborators
ORDER BY collaborations DESC''')

    for record in result:
        print(f'''Artist: {record["name"]}
    N.collaborations: {record["collaborations"]}
    Collaborators: {record["collaborators"]}\n''')

### 6) Stabilire quale sia l'album che ha più tracce nei Wrapped:

In [ ]:
with neo4jInstance.session() as session:
    result = session.run('''MATCH (al:Album)<-[:PART_OF]-(t:Track)<-[:LISTENED_TO]-(u:User)
WITH al.nameAlbum AS album, COUNT(u) as numListeners
ORDER BY numListeners DESC
LIMIT 1
RETURN album''')
    for record in result:
        print(f'''Album più ascoltato: {record["album"]}\n''')

### 7) Stabilire quale sia l'album che ha più tracce per ogni Wrapped:

In [ ]:
with neo4jInstance.session() as session:
    result = session.run('''MATCH (u:User)-[:LISTENED_TO]->(t:Track)-[:PART_OF]->(al:Album) \
                    WITH u, al, COUNT(t) AS numTracks \
                    ORDER BY numTracks DESC \
                    WITH u, COLLECT(al)[0] AS al, MAX(numTracks) as maxTracks \
                    RETURN DISTINCT u.nameUser, al.nameAlbum''')
    for record in result:
        print(f'''User: {record["u.nameUser"]}: 
        Album più ascoltato : {record["al.nameAlbum"]}\n''')

### 8) Per ogni utente, stabilire quale sia l'album che ha più tracce all’interno del proprio Wrapped:

In [ ]:
def most_listened_genre(tx):
    """funzione per trovare il genere più ascoltato"""
    
    result = tx.run('''MATCH (g:Genre)<-[:BELONGS_TO]-(a:Artist)<-[:PERFORMED_BY]-(t:Track)<-[:LISTENED_TO]-(u:User)
WITH g.genres AS genre, COUNT(u) as numListeners
ORDER BY numListeners DESC
LIMIT 1
RETURN genre''')
    return result.single()

In [ ]:
with neo4jInstance.session() as session:
    result = session.read_transaction(most_listened_genre)
    print("Genere più ascoltato:", result["genre"])

### 9) Per ogni utente, stabilire quale sia il genere più ascoltato all’interno del proprio Wrapped:

In [ ]:
def most_listened_genre(tx, nameUser):
    """funzione per trovare il genere più ascoltato da un utente"""
    
    result = tx.run("MATCH (u:User)-[:LISTENED_TO]->(t:Track)-[:PERFORMED_BY]->(a:Artist)-[:BELONGS_TO]->(g:Genre) \
                    WHERE u.nameUser = $nameUser \
                    RETURN g.genres as genre, count(g) as count \
                    ORDER BY count DESC \
                    LIMIT 1", nameUser=nameUser)
    return result.single()

In [ ]:
with neo4jInstance.session() as session:
    # ottieni tutti gli utenti
    users = session.read_transaction(get_all_users)
    
    # per ogni utente, trova il genere più ascoltato
    for user in users:
        result = session.read_transaction(most_listened_genre, user)
        print(f'''Utente: {user} 
        Genere più ascoltato: {result["genre"]}\n''')

### 10) Trovare quali siano gli utenti più simili per quanto riguarda le tracce del Wrapped:

In [ ]:
listo = np.arange(len(tracks_list[0]))

In [ ]:
martol = []
chiara = []
for i in listo:
    martol.append(tracks_list[6][i]['track']['id'])
    chiara.append(tracks_list[5][i]['track']['id'])

In [ ]:
c = pd.Series(chiara)
m = pd.Series(martol)


m=pd.DataFrame(m, columns=['d'])
c=pd.DataFrame(c, columns=['d'])

In [ ]:
len(pd.merge(c, m, "inner"))

In [ ]:
with neo4jInstance.session() as session:
    result = session.run('''MATCH (u1:User)-[:LISTENED_TO]->(t:Track)<-[:LISTENED_TO]-(u2:User)
WHERE u1.nameUser <> u2.nameUser AND u1.nameUser < u2.nameUser
WITH DISTINCT u1.nameUser AS user1, u2.nameUser AS user2, collect(DISTINCT t.nameTrack) AS common_tracks
RETURN user1, user2, common_tracks, size(common_tracks) AS n_tracks
ORDER BY n_tracks DESC
''')
    
    for record in result:
        print(f'''User1: {record["user1"]} 
User2: {record["user2"]} 
    {record["n_tracks"]} numCommonTracks
    tracks: {record["common_tracks"]}\n''')

### 11) Stabilire quale siano gli utenti che hanno più canzoni in comune con tutti gli altri utenti:

In [ ]:
with neo4jInstance.session() as session:
    result = session.run('''MATCH (u1:User)-[:LISTENED_TO]->(t:Track)<-[:LISTENED_TO]-(u2:User)
WITH u1, size(collect(DISTINCT t.nameTrack)) AS common_track
ORDER BY common_track DESC
RETURN u1.nameUser as user, common_track''')

    for record in result:
            print(f'''User: {record["user"]} 
        {record["common_track"]} numCommonTracks with other users\n''')

### 12) Trovare il percorso più vicino tra gli utenti 

In [ ]:
with neo4jInstance.session() as session:
    result = session.run('''MATCH p=shortestPath((u1:User)-[*..]-(u2:User))
WHERE u1 <> u2 and u1>u2
RETURN u1.nameUser AS user1, u2.nameUser AS user2, length(p) as distance
ORDER BY distance DESC''')
    
    for record in result:
        print(f'''User1: {record["user1"]} 
User2: {record["user2"]} 
    {record["distance"]} steps\n''')